# MoneyMinder
Applicazione per la gestione della personal finance, in questo foglio andrò a sistemare le funzioni e le classi per la gestione della personal finance.
ci sarà anche una cella per la dashboard ma dei soli dati, per ora, poi cercherò di mettere tutto insieme.

## Import librerie

In [1]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
import datetime

## Dataset
### Caricare un file


In [4]:
def load_data(file_path):
    # Check if the file extension is CSV
    if not file_path.endswith(".csv"):
        if file_path.endswith(".xlsx"):
            file_path = process_data(file_path, input("source"), input("name"), True)
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    # Convert the "Date" column to datetime format
    df["Date"] = pd.to_datetime(df["Date"])
    # Sort the DataFrame by "Date"
    df = df.sort_values("Date")
    # Convert the "Import" column to float format
    df["Import"] = df["Import"].astype("float")
    # Return the processed DataFrame
    return df

### Processo da applicare a qualsiasi nuovo file in ingresso

In [5]:
def process_data(file_path, source, name, save):
    """
    Questa funzione processa un file di dati Excel e restituisce un DataFrame pandas.
    Parametri:
    - file_path (str): Il percorso del file Excel da processare.
    - source (str): La fonte dei dati.
    - name (str): Il nome della banca.
    Ritorna:
    - df (DataFrame): Il DataFrame processato.
    La funzione esegue le seguenti operazioni solo se source è "bank" e name è "san_paolo":
    1. Carica il file Excel in un DataFrame pandas.
    2. Rimuove le prime righe fino alla prima cella non vuota nella prima colonna.
    3. Imposta la prima riga come nuova intestazione del DataFrame.
    4. Rimuove la prima riga dal DataFrame.
    5. Rinomina gli indici del DataFrame con i valori della nuova intestazione.
    6. Resetta gli indici del DataFrame.
    7. Converte la colonna "Data" in formato datetime.
    8. Rinomina la colonna 'Categoria' in 'Categoria banca'.
    9. Aggiunge le nuove colonne 'Categoria', 'Subcategoria' e 'Commento', inizialmente impostate su None.
    """
    if source == "bank" and name == "san_paolo":
        df = pd.read_excel(file_path)
        # Rimuovi le prime righe fino alla prima cella vuota nella prima colonna
        df = df[df.iloc[:, 0].notna()]
        # Imposta la prima riga come nuova intestazione
        new_header = df.iloc[0]
        # Rimuovi la prima riga dal DataFrame
        df = df[1:]
        # Rinomina gli indici del DataFrame con i valori della nuova intestazione
        df = df.rename(columns=new_header)
        # Resetta gli indici del DataFrame
        df = df.reset_index(drop=True)
        # Rinomina le colonne
        df = df.rename(
            columns={
                "Data": "Date",
                "Categoria ": "Bank Category",
                "Operazione": "Operation",
                "Dettagli": "Details",
                "Conto o carta": "Account or Card",
                "Contabilizzazione": "Accounting",
                "Valuta": "Valute",
                "Importo": "Import",
            }
        )
        # Converte la colonna "Data" in formato datetime
        df["Date"] = pd.to_datetime(df["Date"])
        # Aggiungi le nuove colonne 'Categoria', 'Subcategoria' e 'Commento'
        df[["Category", "Subcategory", "Comment"]] = None, None, None
        # Converti le colonne 'Categoria', 'Subcategoria' e 'Commento' in formato stringa
        df[["Category", "Subcategory", "Comment"]] = df[
            ["Category", "Subcategory", "Comment"]
        ].astype("str")
        # Sort the DataFrame by "Date"
        df = df.sort_values("Date")
        # Convert the "Import" column to float format
        df["Import"] = df["Import"].astype("float")
        if save == True:
            today = datetime.datetime.now()
            filepath = f'C:/Users/Admin/Documents/GitHub/MoneyMinder/data/dataframe_{today.strftime("%Y%m%d_%H%M%S")}.csv'
            df.to_csv(
                filepath,
                index=False,
            )
            return filepath
        else:            
            return df

### Merging file

In [6]:
def merge_and_sort(df1, df2, columns, save):
    """
    Unisce due DataFrame, rimuove i duplicati basandosi su specifiche colonne, converte la colonna "Data" in formato datetime, rimuove le righe con valori mancanti o non validi nella colonna "Data" e ordina il DataFrame in base alla colonna "Data".

    Parametri:
    df1 (pandas.DataFrame): Primo DataFrame da unire.
    df2 (pandas.DataFrame): Secondo DataFrame da unire. fornire il filepath
    columns (list): Lista di colonne su cui basare la rimozione dei duplicati.
    column: lista di colonne che il df2 deve sovrascrivere sul df

    Restituisce:
    df (pandas.DataFrame): DataFrame risultante dopo l'unione, la rimozione dei duplicati, la conversione della colonna "Data" in datetime, la rimozione delle righe con valori mancanti o non validi nella colonna "Data" e l'ordinamento in base alla colonna "Data".
    """
    # Unisci i due DataFrame
    df2 = process_data(df2, "bank", "san_paolo", save=False)
    df = pd.concat([df1, df2], keys=["df1", "df2"])

    # Converte la colonna "Data" in formato datetime
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # Rimuovi i duplicati basandoti su specifiche colonne, mantenendo la prima occorrenza (quella da df1)
    df = df.loc[~df.duplicated(subset=columns, keep="first")]

    # Ordina il DataFrame in base alla colonna "Data"
    df = df.sort_values("Date")

    # Resetta l'indice
    df = df.reset_index(drop=True)

    if save == True:
        df_save(df)
    else:
        return df

In [ ]:
def merge_and_sort2(df1, df2, columns=[
        "Date",
        "Operation",
        "Details",
        "Account or Card",
        "Bank Category",
        "Valute",
        "Import",
    ], save):
    """
    Unisce due DataFrame, rimuove i duplicati basandosi su specifiche colonne, converte la colonna "Data" in formato datetime, rimuove le righe con valori mancanti o non validi nella colonna "Data" e ordina il DataFrame in base alla colonna "Data".

    Parametri:
    df1 (pandas.DataFrame): Primo DataFrame da unire.
    df2 (pandas.DataFrame): Secondo DataFrame da unire. fornire il filepath
    columns (list): Lista di colonne su cui basare la rimozione dei duplicati.
    column: lista di colonne che il df2 deve sovrascrivere sul df

    Restituisce:
    df (pandas.DataFrame): DataFrame risultante dopo l'unione, la rimozione dei duplicati, la conversione della colonna "Data" in datetime, la rimozione delle righe con valori mancanti o non validi nella colonna "Data" e l'ordinamento in base alla colonna "Data".
    """
    # Unisci i due DataFrame
    df2 = process_data(df2, "bank", "san_paolo", save=False)

    dff = df1.merge(df2, how="outer", on=columns)

    # Converte la colonna "Data" in formato datetime
    dff["Date"] = pd.to_datetime(dff["Date"], errors="coerce")

    # Ordina il DataFrame in base alla colonna "Data"
    dff = dff.sort_values("Date")

    # Resetta l'indice
    dff = dff.reset_index(drop=True)

    if save == True:
        df_save(dff)
    else:
        return dff

In [7]:
def df_save(df):
    today = datetime.datetime.now()
    df.to_csv(
        f'C:/Users/Admin/Documents/GitHub/MoneyMinder/data/dataframe_{today.strftime("%Y%m%d_%H%M%S")}.csv',
        index=False,
    )

## Applicazione e comandi utili

In [26]:
process_data(
    "C:/Users/Admin/Documents/GitHub/MoneyMinder/data/lista_completa_13102023.xlsx",
    "bank",
    "san_paolo",
    save=True,
)

'C:/Users/Admin/Documents/GitHub/MoneyMinder/data/dataframe_20231229_170419.csv'

In [8]:
file = "C:/Users/Admin/Documents/GitHub/MoneyMinder/data/lista_completa_15112023.xlsx"
file_csv = (
    "C:/Users/Admin/Documents/GitHub/MoneyMinder/data/dataframe_20231229_170531.csv"
)

In [9]:
df = load_data(file_csv)
df.head()

,Date,Operation,Details,Account or Card,Accounting,Bank Category,Valute,Import,Category,Subcategory,Comment
0,2022-11-02,Trenit S721130bstazione Pinerolo,Effettuato Il 02/11/2022 Alle Ore 0740 Mediant...,Conto 1000/00071615,CONTABILIZZATO,"Trasporti, noleggi, taxi e parcheggi",EUR,-42.00,NaN,NaN,NaN
1,2022-11-05,Lappetito Corso Torino 48 P,LAPPETITO CORSO TORINO 48 P05/112058 Carta N.5...,Conto 1000/00071615,CONTABILIZZATO,Ristoranti e bar,EUR,-14.00,NaN,NaN,NaN
2,2022-11-12,ILIAD ITALIA Roma,EFFETTUATO IL 12/11/2022 ALLE ORE 0209 MEDIANT...,Conto 1000/00071615,CONTABILIZZATO,Cellulare,EUR,-5.99,NaN,NaN,NaN
3,2022-12-01,Trenit S721130bstazione Pinerolo,Effettuato Il 01/12/2022 Alle Ore 0806 Mediant...,Conto 1000/00071615,CONTABILIZZATO,"Trasporti, noleggi, taxi e parcheggi",EUR,-42.00,NaN,NaN,NaN
4,2022-12-07,263806013007357#the Space Cinema Beinasco,Effettuato Il 07/12/2022 Alle Ore 1331 Mediant...,Conto 1000/00071615,CONTABILIZZATO,Spettacoli e musei,EUR,-71.40,NaN,NaN,NaN


In [10]:
df.columns

Index(['Date', 'Operation', 'Details', 'Account or Card', 'Accounting',
       'Bank Category', 'Valute', 'Import', 'Category', 'Subcategory',
       'Comment'],
      dtype='object')

In [30]:
df=merge_and_sort(
    df,
    "C:/Users/Admin/Documents/GitHub/MoneyMinder/data/lista_completa_15112023.xlsx",
    [
        "Date",
        "Operation",
        "Details",
        "Account or Card",
        "Accounting",
        "Bank Category",
        "Valute",
        "Import",
        "Comment",
    ], False
)
df.head()

,Date,Operation,Details,Account or Card,Accounting,Bank Category,Valute,Import,Category,Subcategory,Comment
0,2022-11-02,Trenit S721130bstazione Pinerolo,Effettuato Il 02/11/2022 Alle Ore 0740 Mediant...,Conto 1000/00071615,CONTABILIZZATO,"Trasporti, noleggi, taxi e parcheggi",EUR,-42.00,NaN,NaN,NaN
1,2022-11-05,Lappetito Corso Torino 48 P,LAPPETITO CORSO TORINO 48 P05/112058 Carta N.5...,Conto 1000/00071615,CONTABILIZZATO,Ristoranti e bar,EUR,-14.00,NaN,NaN,NaN
2,2022-11-12,ILIAD ITALIA Roma,EFFETTUATO IL 12/11/2022 ALLE ORE 0209 MEDIANT...,Conto 1000/00071615,CONTABILIZZATO,Cellulare,EUR,-5.99,NaN,NaN,NaN
3,2022-12-01,Trenit S721130bstazione Pinerolo,Effettuato Il 01/12/2022 Alle Ore 0806 Mediant...,Conto 1000/00071615,CONTABILIZZATO,"Trasporti, noleggi, taxi e parcheggi",EUR,-42.00,NaN,NaN,NaN
4,2022-12-07,263806013007357#the Space Cinema Beinasco,Effettuato Il 07/12/2022 Alle Ore 1331 Mediant...,Conto 1000/00071615,CONTABILIZZATO,Spettacoli e musei,EUR,-71.40,NaN,NaN,NaN


In [11]:
df_save(df)

In [ ]:
dff = df1.merge(df2, how="outer", on=["colonna1", "colonna2", ...])

In [12]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 175 entries, 0 to 174
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             175 non-null    datetime64[ns]
 1   Operation        175 non-null    object        
 2   Details          175 non-null    object        
 3   Account or Card  175 non-null    object        
 4   Accounting       175 non-null    object        
 5   Bank Category    175 non-null    object        
 6   Valute           175 non-null    object        
 7   Import           175 non-null    float64       
 8   Category         0 non-null      float64       
 9   Subcategory      0 non-null      float64       
 10  Comment          0 non-null      float64       
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 16.4+ KB
